In [6]:
import subprocess
import shutil
import time
import win32com.client
import pythoncom
import argparse
from calendar import monthrange
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import Correos

In [7]:
def sapConnection(cSap):
    path=r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe"
    subprocess.Popen(path)
    time.sleep(5)
    SapGuiAuto = win32com.client.GetObject("SAPGUI")
    application = SapGuiAuto.GetScriptingEngine

    try:
        connection = application.Children(0)

        i=0
        print("Cerrando {} sesion(es) activa(s)".format(int(connection.children.count)))
        while int(connection.children.count) > 0 and i <5:
            session = connection.Children(0)
            session.findbyid("wnd[0]").close()
            session.findById("wnd[1]/usr/btnSPOP-OPTION1").press()
            i=i+1
                
    except Exception as e:
        pass
        
    connection = application.OpenConnection("RISE - ERP Produccion")
    session = connection.Children(0)
    session.findById("wnd[0]").maximize()
    session.findById("wnd[0]/usr/txtRSYST-BNAME").text = "1030611534"
    session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = "Tebis.1030611534"
    session.findById("wnd[0]").sendVKey(0)
    return session

In [8]:
session=sapConnection(False)

In [9]:
df=pd.read_excel("Plano tarifas Ad.xlsx",converters={"Centro de coste":str,"Clase Actividad":str})

In [10]:
for i in df.values:
    session.findById("wnd[0]/tbar[0]/okcd").text = "kp26"
    session.findById("wnd[0]").sendVKey(0)
    
    try:
        session.findById("wnd[1]/usr/sub:SAPLSPO4:0300/ctxtSVALD-VALUE[0,21]").text = "CO10"
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
    except:
        pass
    
    session.findById("wnd[0]/tbar[1]/btn[32]").press()
    session.findById("wnd[0]/tbar[1]/btn[32]").press()

    session.findById("wnd[0]/usr/sub:SAPLKPP0:1000/ctxtKPP0B-VALUE[0,21]").text = "0"
    session.findById("wnd[0]/usr/sub:SAPLKPP0:1000/ctxtKPP0B-VALUE[1,21]").text = "1"
    session.findById("wnd[0]/usr/sub:SAPLKPP0:1000/ctxtKPP0B-VALUE[2,21]").text = "12"
    session.findById("wnd[0]/usr/sub:SAPLKPP0:1000/ctxtKPP0B-VALUE[3,21]").text = "2023"
    session.findById("wnd[0]/usr/sub:SAPLKPP0:1000/ctxtKPP0B-VALUE[5,21]").text = i[0]
    session.findById("wnd[0]/usr/sub:SAPLKPP0:1000/ctxtKPP0B-VALUE[8,21]").text = i[1]

    session.findById("wnd[0]").sendVKey(0)

    session.findById("wnd[0]/tbar[1]/btn[6]").press()

    for j in range(7,12):
        session.findById("wnd[0]/usr/tblSAPLKPP2TABCONTROL/txtZ-BDC06[5,{}]".format(j)).text = i[2]

    session.findById("wnd[0]").sendVKey(0)

    session.findById("wnd[0]/tbar[0]/btn[11]").press()
    session.findById("wnd[0]/tbar[0]/btn[12]").press()
    